In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
#Extrae posiciones de inicio, fin y tags de diccionarios en events
def extraer_posiciones_events(location):
    with open(location,'r',encoding='utf8') as json_file:
        data=json.load(json_file)
    for parada in data: 
        location=parada.pop("positions")
        parada['x_inicio']=location[0]['x']
        parada['y_inicio']=location[0]['y']
        if len(location)>1:
            parada['x_fin']=location[1]['x']
            parada['y_fin']=location[1]['y']
        if 'tags' in parada.keys():
            etiquetas=parada.pop("tags")
            for i in range(0,len(etiquetas)):
                tags_ind='tag_'+str(i)
                parada[tags_ind]=etiquetas[i]['id']
    df=pd.DataFrame(data)
    return df
#Extrae area de datos teams
def extraer_area(location):
    with open(location,'r',encoding='utf8') as json_file:
        data=json.load(json_file)
    for parada in data: 
        area=parada.pop('area')
        parada['country']=area['name']
        parada['id']=area['id']
        parada['alpha3code']=area['alpha3code']
    df=pd.DataFrame(data)
    return df

def separar_jugadas_2(df):

  team1 = df.teamId.unique()[0]
  team2 = df.teamId.unique()[1]

  #eliminar eventos que no queremos

  df1 = df.drop(df[(df['tag_0'] == 701) | # quitar duelos perdidos. Parece que solo aparecen en los 
                  (df['tag_1'] == 701) |
                  (df['tag_2'] == 701) |
                  (df['tag_3'] == 701) |
                  (df['tag_4'] == 701) |
                  (df['tag_5'] == 701)].index)
  
  df1 = df1.drop(df1[((df1['eventName'] == 'Duel') & (df1['teamId'].shift(-1) == df1['teamId'].shift(1)))].index) #Team - Duel - Team

     
  df1 = df1[(df1['eventName'] != "Save attempt")]

  df1['playId'] = ( ((df1['teamId'] != df1['teamId'].shift(2)) &  ((df1['teamId'] != df1['teamId'].shift(1))) |  #No las cuenta si alguna de las últimas 2 acciones son del otro equipo
                       (df1['eventName'].shift(1) == 'Interruption') | #sumar si el evento pasado es un interruption
                       (df1['matchPeriod'].shift(1) != df1['matchPeriod']) | #Asegura que en la transn 1H-2H se genere nueva jugada 
                       (df1['eventName'] == 'Free Kick') #Asegurar que los tiros libres inicien una nueva jugada
                       )).cumsum()                     

  (df1.loc[
      ((df1['teamId'] != df1['teamId'].shift(1)) & 
       (df1['teamId'] != df1['teamId'].shift(-1))),'playId']) = 0
    
  df['playId'] = (pd.concat((df,df1['playId']),axis=1))['playId'].fillna(0)

  return df

#Partido España vs Portugal
#Prueba
location="/Users/robertodelriosalgado/Desktop/proyecto verano 2022/events_World_Cup.json"

df1 = extraer_posiciones_events(location)
dfmid = df1[df1['matchId'] == 2057960]

dfjugadas = separar_jugadas_2(dfmid)

/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
dfjugadascopy=dfjugadas

In [4]:
dfjugadascopy=dfjugadascopy.assign(pass_dist = [float(0)]*len(dfjugadascopy),shortpass_good=[int(0)]*len(dfjugadascopy),
                                   longpass_good=[int(0)]*len(dfjugadascopy),shortpass_bad=[int(0)]*len(dfjugadascopy),
                                   longpass_bad=[int(0)]*len(dfjugadascopy), key_passes=[int(0)]*len(dfjugadascopy),
                                   assists=[int(0)]*len(dfjugadascopy))
dfjugadascopy=dfjugadascopy.assign(shot_dist = [float(0)]*len(dfjugadascopy), 
                                   shots_on_goal=[int(0)]*len(dfjugadascopy),shots_off_goal=[int(0)]*len(dfjugadascopy), 
                                   goals=[int(0)]*len(dfjugadascopy), kph_goals=[float(0)]*len(dfjugadascopy))

dfjugadascopy=dfjugadascopy.assign(offsides=[int(0)]*len(dfjugadascopy))

dfjugadascopy=dfjugadascopy.assign(fouls=[int(0)]*len(dfjugadascopy), violent_fouls=[int(0)]*len(dfjugadascopy), 
                                   simulation_fouls=[int(0)]*len(dfjugadascopy), penalty_fouls=[int(0)]*len(dfjugadascopy),
                                  fouls_defensive_third=[int(0)]*len(dfjugadascopy),fouls_against=[int(0)]*len(dfjugadascopy),
                                  fouls_against_attacking_third=[int(0)]*len(dfjugadascopy), yellow_cards=[int(0)]*len(dfjugadascopy),
                                  red_cards=[int(0)]*len(dfjugadascopy))
dfjugadascopy=dfjugadascopy.assign(accelerations=[int(0)]*len(dfjugadascopy) , acceleration_kph=[float(0)]*len(dfjugadascopy))

dfjugadascopy=dfjugadascopy.assign(goals_team_9905=[int(0)]*len(dfjugadascopy), goals_team_1598=[int(0)]*len(dfjugadascopy))



In [ ]:
#lol=dfjugadas
#team1='goals_'+str(list(set(list(lol['teamId'].values)))[0])
#team2=list(set(list(lol['teamId'].values)))[1]
#dfjugadascopy['goals_team_9905'].name.split('_')[-1]

In [5]:

for i,jugada in dfjugadascopy.iterrows():
    tags=jugada.tolist()[14:20]
    if (float(101) in tags) and (jugada['eventId']!=9):#goals
            if jugada['subEventId']!=35: #penalties not included
                distanciax=(abs(jugada['x_inicio'] - 100)*110)/100
                distanciay=(abs(jugada['y_inicio'] - 50)*75)/100
                distancia_km=np.sqrt(distanciax**2+distanciay**2)/1000
                diff_time_hour=abs(dfjugadascopy.loc[i+1]['eventSec']-jugada['eventSec'])/3600
                dfjugadascopy['kph_goals'][i]=distancia_km/diff_time_hour
            dfjugadascopy['goals'][i]=1
            dfjugadascopy['shots_on_goal'][i]=1
            goal_id=jugada['playId']
            if goal_id!=0: #assists that might not be documented
                df_goal_play=dfjugadascopy[dfjugadascopy['playId']==goal_id]
                if (df_goal_play.iloc[-2]['eventId'] in [3,1,8] and len(df_goal_play)>=2 
                    and df_goal_play.iloc[-2]['playerId']!=jugada['playerId'] and 
                    int(df_goal_play.iloc[-1].name)==int(df_goal_play.iloc[-2].name)+1):
                    assist_player_row=int(df_goal_play.iloc[-2].name)
                    dfjugadascopy['assists'][assist_player_row] = 1
                    if dfjugadascopy['subEventId'][assist_player_row] not in [30,31,32,34,80,82,83,84,85,86]:
                        distanciax=(abs(dfjugadascopy['x_inicio'][assist_player_row]-dfjugadascopy['x_fin'][assist_player_row])*110)/100 #110x75
                        distanciay=(abs(dfjugadascopy['y_inicio'][assist_player_row]-dfjugadascopy['y_fin'][assist_player_row])*75)/100
                        distancia=np.sqrt(distanciax**2+distanciay**2)
                        dfjugadascopy['pass_dist'][i]=distancia
                        if distancia <= 10:
                            dfjugadascopy['shortpass_good'][i]=1
                        else:
                            dfjugadascopy['longpass_good'][i]=1 
                            
    if (float(301) in tags) and (jugada['eventId']!=9): #assists
            dfjugadascopy['assists'][i]=1
            if jugada['subEventId'] not in [30,31,32,34,80,82,83,84,85,86]:
                distanciax=(abs(jugada['x_inicio']-jugada['x_fin'])*110)/100 #110x75
                distanciay=(abs(jugada['y_inicio']-jugada['y_fin'])*75)/100
                distancia=np.sqrt(distanciax**2+distanciay**2)
                dfjugadascopy['pass_dist'][i]=distancia
                if distancia <= 10:
                    dfjugadascopy['shortpass_good'][i]=1
                else:
                    dfjugadascopy['longpass_good'][i]=1  
                     
    if jugada['subEventId'] in [30,31,32,34,80,82,83,84,85,86]:  #passes
        distanciax=(abs(jugada['x_inicio']-jugada['x_fin'])*110)/100
        distanciay=(abs(jugada['y_inicio']-jugada['y_fin'])*75)/100
        distancia=np.sqrt(distanciax**2+distanciay**2)
        dfjugadascopy['pass_dist'][i]=distancia
        if distancia <= 10 and (float(1801) in tags):
            dfjugadascopy['shortpass_good'][i]=1
        if float(302) in tags:
            dfjugadascopy['key_passes'][i]=1
        elif distancia <= 10 and (float(1802) in tags):
            dfjugadascopy['shortpass_bad'][i]=1
        elif distancia > 10 and (float(1802) in tags):
            dfjugadascopy['longpass_bad'][i]=1
        elif distancia > 10 and (float(1801) in tags):
            dfjugadascopy['longpass_good'][i]=1
        
    elif jugada['subEventId'] in [33,35,100]: #shots
        distanciax=(abs(jugada['x_inicio'] - 100)*110)/100
        distanciay=(abs(jugada['y_inicio'] - 50)*75)/100
        distancia=np.sqrt(distanciax**2+distanciay**2)
        dfjugadascopy['shot_dist'][i]=distancia
        if (float(1801) in tags and float(101) not in tags):
            dfjugadascopy['shots_on_goal'][i]=1
        elif (float(1802) in tags):
            dfjugadascopy['shots_off_goal'][i]=1 
            
    if jugada['eventId']==6: #offsides
        dfjugadascopy['offsides'][i]=1
        
    if jugada['subEventId'] in [20,21,25,27]: #faltas
        dfjugadascopy['fouls'][i]=1
        if jugada['subEventId']==27:
            dfjugadascopy['violent_fouls'][i]=1
        if jugada['subEventId']==25:
            dfjugadascopy['simulation_fouls'][i]=1
        if jugada['x_fin']<=16 and (jugada['y_fin']>=19 or jugada['y_fin']<=81):
            dfjugadascopy['penalty_fouls'][i]=1
        if jugada['x_fin']<=33:
            dfjugadascopy['fouls_defensive_third'][i]=1
        if jugada['subEventId'] in [20,27]:
            foul_against_player1=i-2
            foul_against_player2=i-1
            if jugada['playerId']==dfjugadascopy['playerId'][foul_against_player1]:
                dfjugadascopy['fouls_against'][foul_against_player2]=1
                if dfjugadascopy['x_fin'][foul_against_player2]>=66:
                    dfjugadascopy['fouls_against_attacking_third'][foul_against_player2]=1
            elif jugada['playerId']==dfjugadascopy['playerId'][foul_against_player2]:
                dfjugadascopy['fouls_against'][foul_against_player1]=1
                if dfjugadascopy['x_fin'][foul_against_player1]>=66:
                    dfjugadascopy['fouls_against_attacking_third'][foul_against_player1]=1
        if float(1702) in tags:
            dfjugadascopy['yellow_cards'][i]=1
        if float(1701) in tags:
            dfjugadascopy['red_cards'][i]=1
            
    if jugada['eventId']==7: #others on the ball
        if jugada['subEventId']==70: #acceleration (malos datos, nada confiable)
            distanciax=(abs(jugada['x_inicio']-jugada['x_fin'])*110)/100 #110x75
            distanciay=(abs(jugada['y_inicio']-jugada['y_fin'])*75)/100
            distancia_km=np.sqrt(distanciax**2+distanciay**2)/1000
            diff_time_hour=abs(dfjugadascopy.loc[i+1]['eventSec']-jugada['eventSec'])/3600
            dfjugadascopy['acceleration_kph'][i]=distancia_km/diff_time_hour
    
    

            
        
        
        

/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/robertodelri

In [6]:
indices_goles=list(dfjugadascopy[dfjugadascopy['goals']!=0].index)
for i,jugada in dfjugadascopy.iterrows():
    if i > list(dfjugadascopy.index)[0]:
        goals_team_1598=dfjugadascopy['goals_team_1598'][i-1]
        goals_team_9905=dfjugadascopy['goals_team_9905'][i-1]
        if i in indices_goles:
            if dfjugadascopy['goals_team_9905'].name.split('_')[-1]==str(jugada['teamId']):
                dfjugadascopy['goals_team_9905'][i]=goals_team_9905+1
                dfjugadascopy['goals_team_1598'][i]=goals_team_1598
            else:
                dfjugadascopy['goals_team_1598'][i]=goals_team_1598+1
                dfjugadascopy['goals_team_9905'][i]=goals_team_9905
        else:
            dfjugadascopy['goals_team_1598'][i]=goals_team_1598
            dfjugadascopy['goals_team_9905'][i]=goals_team_9905

/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/robertodelriosalgado/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [ ]:
list(dfjugadascopy.index)[0]

In [ ]:
dfjugadascopy.loc[10678]
#1.822
#

In [ ]:
time=(dfjugadascopy.loc[9650]['eventSec']-dfjugadascopy.loc[9649]['eventSec'])#/3600
dist=dfjugadascopy.loc[9649]['shot_dist']/1000
print(time)

In [ ]:
#tag=['tag_0','tag_1','tag_2','tag_3','tag_4','tag_5','playId','eventId']
#lol=dfjugadascopy[dfjugadascopy['playerId']==3304]#[tag]Degea
#lol[lol['fouls_against']==1][tag]
dfjugadas[dfjugadas['subEventId']==13]
#dfjugadascopy[dfjugadascopy['tag_0']==1702]#['playId']
#dfjugadascopy[dfjugadascopy['assists']==1]
#dfjugadas.head(10)  puedo hacer numero de toques y tiempo por jugada, sacar ritmo, promedio de cuanto tiempo dura con el balon

In [48]:
dfjugadas.loc[11271]

eventId                             1
subEventName    Ground defending duel
playerId                         8126
matchId                       2057960
eventName                        Duel
teamId                           9905
matchPeriod                        2H
eventSec                      2965.94
subEventId                         12
id                          258702737
x_inicio                           26
y_inicio                           14
x_fin                              37
y_fin                               0
tag_0                             702
tag_1                            1801
tag_2                             NaN
tag_3                             NaN
tag_4                             NaN
tag_5                             NaN
playId                            230
Name: 11271, dtype: object

In [45]:
dfjugadas[dfjugadas['eventId']==1]

,eventId,subEventName,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,...,y_inicio,x_fin,y_fin,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,playId
9600,1,Air duel,8317,2057960,Duel,1598,1H,27.983234,10,258700827,...,92,63.0,92.0,701.0,1802.0,NaN,NaN,NaN,NaN,0.0
9601,1,Air duel,28907,2057960,Duel,9905,1H,28.196555,10,258700816,...,8,37.0,8.0,703.0,1801.0,NaN,NaN,NaN,NaN,2.0
9602,1,Ground attacking duel,243588,2057960,Duel,9905,1H,31.011317,11,258700819,...,8,23.0,0.0,702.0,1801.0,NaN,NaN,NaN,NaN,2.0
9603,1,Ground defending duel,3304,2057960,Duel,1598,1H,31.537988,12,258700830,...,92,77.0,100.0,702.0,1801.0,NaN,NaN,NaN,NaN,0.0
9606,1,Ground defending duel,8317,2057960,Duel,1598,1H,40.115834,12,258700833,...,93,67.0,90.0,504.0,703.0,1801.0,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11236,1,Air duel,3346,2057960,Duel,1598,2H,2802.480457,10,258702849,...,44,9.0,46.0,703.0,1801.0,NaN,NaN,NaN,NaN,0.0
11270,1,Ground attacking duel,3840,2057960,Duel,1598,2H,2965.626861,11,258702844,...,86,63.0,100.0,702.0,1801.0,NaN,NaN,NaN,NaN,229.0
11271,1,Ground defending duel,8126,2057960,Duel,9905,2H,2965.940856,12,258702737,...,14,37.0,0.0,702.0,1801.0,NaN,NaN,NaN,NaN,230.0
11273,1,Ground attacking duel,70403,2057960,Duel,9905,2H,2994.415303,11,258702741,...,4,0.0,0.0,701.0,1802.0,NaN,NaN,NaN,NaN,0.0


In [40]:
list_subevent1=[10,11,12,13,20,21,22,23,24,25,26,27,30,31,32,33,34,35,36,40,50,51,60,70,71,72,80,81,82,83,84,85
              ,86,90,91,100]
list_event=[1,2,3,4,5,6,7,8,9,10]
list_tags=[1801,1802]

tag=['tag_0','tag_1','tag_2','tag_3','tag_4','tag_5','playId']
columnspass=dfjugadascopy.columns[21:28].tolist()
#columnspass=['playerId']+columnspass
list_eventName=list(dict.fromkeys(dfjugadascopy.eventName.values.tolist()))
columnsshot=dfjugadascopy.columns[28:33].tolist()

In [41]:
dicc_jugador={}
for i in dfjugadascopy['playerId']:
    dicc_jugador[i]={}
    for j in list_eventName:
        dicc_jugador[i][j]={}

In [42]:
for playerId in dicc_jugador:
    player=dicc_jugador[playerId]
    dfplayer=dfjugadascopy.loc[dfjugadascopy["playerId"] == playerId]
    for events in player:
        if events=='Pass':
            dfplayerpasses=dfplayer[columnspass]
            for column in dfplayerpasses.columns:
                if not column=='pass_dist':
                    counts=dfplayer.sum()
                    player[events][column]=counts[column]
                else:
                    dist_list = dfplayerpasses[column].tolist()
                    dist_list1 = dfplayerpasses['longpass_good'].tolist()
                    dist_list2 = dfplayerpasses['shortpass_good'].tolist()
                    dist_list_filtered = [dist_list[i] for i in range(len(dist_list)) if (dist_list1[i]==1 or dist_list2[i]==1)] 
                    player[events]['pass_dist_compleated']=dist_list_filtered
            player[events]['completed_passes']=player[events]['shortpass_good']+player[events]['longpass_good']            
            player[events]['attemped_passes']=(player[events]['completed_passes']+player[events]['shortpass_bad']+player[events]['longpass_bad'])
            player[events]['%_passes']=player[events]['completed_passes']/player[events]['attemped_passes']
        if events=='Shot':
            dfplayershots=dfplayer[columnsshot]
            for column in dfplayershots.columns:
                if  column =='kph_goals': 
                    dist_list = dfplayershots['kph_goals'].tolist()
                    dist_list_filtered = [dist_list[i] for i in range(len(dist_list)) if (dist_list[i]!=0)] 
                    player[events]['kph_goals']=dist_list_filtered
                elif column=='shot_dist':
                    dist_list = dfplayershots['shot_dist'].tolist()
                    dist_list1 = dfplayershots['shots_on_goal'].tolist()
                    dist_list_filtered = [dist_list[i] for i in range(len(dist_list)) if (dist_list1[i]==1)] 
                    player[events]['shot_on_goal_dist']=dist_list_filtered
                else:
                    counts=dfplayer.sum()
                    player[events][column]=counts[column]        
            player[events]['attemped_shots']=player[events]['shots_on_goal'] + player[events]['shots_off_goal']
            if player[events]['attemped_shots']!=0:
                player[events]['%_shots_on_goal']=player[events]['shots_on_goal']/ player[events]['attemped_shots']
                player[events]['%_goals_per_attempt']=player[events]['goals']/player[events]['attemped_shots']
            else:
                player[events]['%_shots_on_goal']=0
                player[events]['%_goals_per_attempt']=0
        if events=='Offside':
            player[events]['offsides']=dfplayer['offsides'].sum()
        
        if events=='Foul':
            player[events]['total_fouls']=dfplayer['fouls'].sum()
            player[events]['total_violent_fouls']=dfplayer['violent_fouls'].sum()
            player[events]['total_simulation_fouls']=dfplayer['simulation_fouls'].sum()
            player[events]['penalty_fouls']=dfplayer['penalty_fouls'].sum()
            player[events]['fouls_defensive_third']=dfplayer['fouls_defensive_third'].sum()
            player[events]['fouls_against']=dfplayer['fouls_against'].sum()
            player[events]['fouls_against_attacking_third']=dfplayer['fouls_against_attacking_third'].sum()
            player[events]['yellow_cards']=dfplayer['yellow_cards'].sum()
            player[events]['red_cards']=dfplayer['red_cards'].sum()
        
        if events=='Others on the ball':
            dist_list = dfplayer['acceleration_kph'].tolist()
            dist_list_filtered = [dist_list[i] for i in range(len(dist_list)) if (dist_list[i]!=0)] 
            player[events]['acceleration_kph']=dist_list_filtered
            player[events]['accelerations']=len(dfplayer[dfplayer['subEventId']==70])
            

In [ ]:
dfplayer=dfjugadascopy.loc[dfjugadascopy["playerId"] == 3322]
len(dfplayer[dfplayer['subEventId']==70])

In [43]:
dicc_jugador[3322]

{'Pass': {'pass_dist_compleated': [8.926925562588725,
   17.74323814865821,
   13.94596715900335,
   9.928368446023748,
   13.020752666416794,
   34.0156214113457,
   19.52331938989884,
   45.10623571081941,
   12.2,
   24.790169422575556,
   7.5,
   3.1953090617340916,
   15.704219178297278,
   15.20690632574555,
   13.043101624997023,
   34.0743671988197,
   7.106335201775948,
   10.72800074571213,
   11.562439189029277,
   4.94191258522447,
   17.74323814865821,
   3.384154251803543,
   21.74948275247023,
   34.23813663153998,
   11.853269591129697,
   20.586646157157315,
   18.603023947734947,
   14.475928294931556,
   11.10180165558726],
  'shortpass_good': 7,
  'longpass_good': 22,
  'shortpass_bad': 0,
  'longpass_bad': 0,
  'key_passes': 1,
  'assists': 0,
  'completed_passes': 29,
  'attemped_passes': 29,
  '%_passes': 1.0},
 'Duel': {},
 'Free Kick': {},
 'Others on the ball': {'acceleration_kph': [27.454099114542387,
   32.15135176159217,
   4.593374458064539,
   8.063844570

In [ ]:
dicc_jugador[3304]

In [ ]:
dicc_jugador[3346] #sergio busquets una falta wyscout se la contó a nacho 

In [ ]:
#sacar tiempos de jugadas y numero de toques y sacar promedios 

In [44]:
dicc_equipo={}
for i in list(set(dfjugadascopy['teamId'].tolist())):
    dicc_equipo[i]={}
    dicc_equipo[i]['Jugadas']={}
    dfequipo=dfjugadascopy[dfjugadascopy['teamId']==i]
    lista_tuplas=[]
    for j in list(set(dfequipo['playId'].tolist())):
        dfjugada=dfjugadascopy[dfjugadascopy['playId']==j]
        if j!=0 and len(dfjugada)>2: #tuple(#jugada, #de acciones, tiempo, #pases, #tiros)
            num_acciones=len(dfjugada)
            diff_tiempo=dfjugada.tail(1)['eventSec'].values[0]-dfjugada.head(1)['eventSec'].values[0]
            num_pases=len(dfjugada[dfjugada['eventId']==8])
            num_tiros=len(dfjugada[dfjugada['eventId']==10])
            if num_pases != 0 or num_tiros!=0:
                tupla=(j,num_acciones,diff_tiempo,num_pases,num_tiros)
                lista_tuplas.append(tupla)
    total_acciones=[]
    total_dif_tiempo=[]
    total_num_pases=[]
    total_num_tiros=[]
    for t in lista_tuplas:
        total_acciones.append(t[1])
        total_dif_tiempo.append(t[2])
        total_num_pases.append(t[3])
        total_num_tiros.append(t[4])
    dicc_equipo[i]['Jugadas']['info']=lista_tuplas
    dicc_equipo[i]['Jugadas']['prom']={'total_jugada':len(lista_tuplas),'acciones':np.mean(total_acciones),'dif_tiempo_seg':np.mean(total_dif_tiempo)
                                      , 'num_pases':np.mean(total_num_pases),'num_tiros':np.mean(total_num_tiros), 'acciones_por_min':np.mean(total_acciones)/(np.mean(total_dif_tiempo)/60)
                                      , 'pases_por_min':np.mean(total_num_pases)/(np.mean(total_dif_tiempo)/60)}
    


In [30]:
print(dicc_equipo[1598]['Jugadas']['prom'])
print(dicc_equipo[9905]['Jugadas']['prom'])

{'total_jugada': 70, 'acciones': 12.228571428571428, 'dif_tiempo_seg': 27.492789171428548, 'num_pases': 10.32857142857143, 'num_tiros': 0.11428571428571428, 'acciones_por_min': 26.687517266410595, 'pases_por_min': 22.540975448148206}
{'total_jugada': 60, 'acciones': 6.683333333333334, 'dif_tiempo_seg': 15.849180733333272, 'num_pases': 5.25, 'num_tiros': 0.05, 'acciones_por_min': 25.300992319220335, 'pases_por_min': 19.874844340534676}
